In [1]:
import pandas as pd

import numpy as np
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import _tree
import random

from skopt.space import Real, Categorical, Integer
random.seed(42)

In [2]:
def _sample_search_space(param_space, categorical_columns, num_samples = 1000):
    """
        Uniformly sample the search space for generating explanation dataset. Maintains the original order of search space.
        Sensitive to order: place categoricals first and everything else later.

        Args:
        - param_space (param space): parameter space
        - categorical_columns (list): list of categorical variables
        - num_samples (int): number of generated samples

        Returns:
        - new_data (numpy array): the augmented dataset
        """
    samples = []
    
    for _ in range(num_samples):
        sample = [param.rvs() for param in param_space]
        samples.append(sample)
    
    # Create a DataFrame with the samples
    param_names = [param.name for param in param_space]
    df = pd.DataFrame(samples, columns=param_names)
    
    # One-hot encode categorical variables and add dynamic prefixes
    encoded_columns = []
    for col in categorical_columns:
        prefix = f'categorical_{col}'
        encoded = pd.get_dummies(df[col], prefix=prefix).astype(float)
        encoded_columns.extend(encoded.columns)
        df = pd.concat([df.drop(col, axis=1), encoded], axis=1)
    
    # Rearrange columns: categorical columns in front, followed by non-categorical columns
    ordered_columns = []
    for col in param_space:
        if col.name in categorical_columns:
            ordered_columns.extend([col_name for col_name in df.columns if col_name.startswith(f'categorical_{col.name}')])
        elif col.name in df.columns:
            ordered_columns.append(col.name)
    
    # Now you have the DataFrame with categorical variables in the front and non-categorical variables in their original order
    df = df[ordered_columns]
    
    # Convert numerical variables from lists to floats
    for col in param_names:
        if col not in categorical_columns:
            df[col] = df[col]
    df = df.to_numpy().astype(np.float32)
    return df

In [3]:
df = pd.read_csv('californiahousing_MLPR.csv', index_col=0)
ye = df['0']
#ye = (ye - np.min(ye)) / (np.max(ye) -np.min(ye))
y = ye.to_numpy()

Xe = df.drop(columns='0')
dum1 = pd.get_dummies(Xe['optimizer'],prefix = 'categorical_optimizer')
dum2 = pd.get_dummies(Xe['activation'],prefix = 'categorical_activation')
dum3 = pd.get_dummies(Xe['loss'],prefix = 'categorical_loss')
dum4 = pd.get_dummies(Xe['batch_size'],prefix = 'categorical_batchsize')
Xe = Xe.drop(columns = ['optimizer','activation','loss','batch_size'])
Xe = pd.concat([dum1, dum2, dum3, dum4, Xe], axis=1)

param_space = [
    Categorical(categories=['adam', 'rmsprop'], name='optimizer'),
    Categorical(categories=['relu', 'tanh'], name='activation'),
    Categorical(categories=['mae', 'mse'], name='loss'),
    Categorical(categories=['16', '32', '64'], name='batch_size'),
    Integer(low=20, high=50, name='epochs'),
    Integer(low=10, high=100, name='neurons'),
    Integer(low=3, high=20, name='patience')
]

# Define the categorical columns
categorical_columns = ['optimizer', 'activation', 'loss', 'batch_size']


print (Xe.iloc[np.argmin(ye.values)])
print (np.min(ye.values))

categorical_optimizer_adam        0
categorical_optimizer_rmsprop     1
categorical_activation_relu       1
categorical_activation_tanh       0
categorical_loss_mae              0
categorical_loss_mse              1
categorical_batchsize_16          0
categorical_batchsize_32          0
categorical_batchsize_64          1
epochs                           30
neurons                          91
patience                         20
Name: 4, dtype: int64
0.5343675838170645


In [4]:
import GPy

def _train_gp(x_train, y_train):
    """
        Train a GPR model from Gpy library.

        Args:
        - X_train (numpy array): input data
        - Y_train (numpy array): ground truth data
        - n_samples (int): number of times to sample the posterior of the trained gp
        - sample_points (numpy array): new input X locations where the posterior would be sampled

        Returns:
        - model (trained gp model): trained gp model
        - mean_y (numpy array): mean of the Y from the sampled posterior
        - std_y (numpy array): std dev of the Y from the sampled posterior
        - sum_log_likelihoods (numpy array): returns the likelihood of observing
          each input point given the trained model
    """
    kernel = GPy.kern.RBF(input_dim=x_train.shape[1])
    model = GPy.models.GPRegression(x_train, y_train, kernel)
    model.optimize()
    return model

def augment_with_noise(data, n_samples=1000):
    """
    Add noise to the input data to create new points and generate a new dataset
    with the desired number of samples.

    Args:
    - data (numpy array): the input data
    - num_samples (int): the number of samples to generate
    - noise_factor (float): the magnitude of the noise to add to the data (default: 0.01)

    Returns:
    - new_data (numpy array): the augmented dataset
    """
    n_points, n_dims = data.shape
    new_data = np.empty((n_samples, n_dims))  # Create a new array for the augmented data

    for i in range(n_samples):
        noise = np.random.uniform(low=-0.01, high=0.01, size=n_dims)
        new_data[i] = data[np.random.randint(n_points)] + noise

    return new_data
    
model = _train_gp(Xe.to_numpy(), y.reshape(-1, 1))
test_set = _sample_search_space(param_space, categorical_columns, num_samples = 10000)
test_set = augment_with_noise(test_set)
mean_y, var_y = model.predict_noiseless(test_set, full_cov=False)
flattened_mean = [item for sublist in mean_y for item in sublist]
flattened_var = [item for sublist in var_y for item in sublist]
std = np.sqrt(flattened_var)

samp = _sample_search_space(param_space, categorical_columns, num_samples = 1000)
samp = augment_with_noise(samp)
yee, var = model.predict_noiseless(samp, full_cov=False)
yee = np.squeeze(yee, axis=1)

In [5]:
from rulekit import RuleKit
from rulekit.regression import RuleRegressor
from rulekit.params import Measures
RuleKit.init()

reg = RuleRegressor(
    induction_measure=Measures.C2,
    pruning_measure=Measures.C2,
    voting_measure=Measures.C2,
)
reg.fit(samp, yee)
predictions = reg.predict(test_set)

ru = []
for rule in reg.model.rules:
    ru.append(str(rule))
    print (rule)

print(len(ru))

from itertools import combinations

# Given rule list in the new representation
rule_list = ru

# Function to extract features from a rule
def extract_features(rule):
    # Extract only the part before "THEN" and split by "AND"
    features = set(rule.split('IF')[1].split(" THEN")[0].split(" AND"))
    return features

# Function to calculate Jaccard similarity between two rules
def jaccard_similarity(rule1, rule2):
    features1 = extract_features(rule1)
  
    features2 = extract_features(rule2)
   
    intersection = len(features1.intersection(features2))
    union = len(features1.union(features2))
    
    if union == 0:
        return 0.0  # Handle the case where both sets are empty
    else:
        return intersection / union

# Calculate Jaccard similarity between all pairs of rules and collect them
similarities = []
for rule1, rule2 in combinations(rule_list, 2):
    similarity = jaccard_similarity(rule1, rule2)
    similarities.append(similarity)

# Calculate the mean Jaccard similarity for the entire rule set
mean_similarity = sum(similarities) / len(similarities)

print(f"Mean Jaccard Similarity for the Rule Set: {mean_similarity:.2f}")


lower_bounds = [flattened_mean[i] - 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]
upper_bounds = [flattened_mean[i] + 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]

# Initialize a count for predictions within bounds
count_within_bounds = sum(1 for prediction, lower_bound, upper_bound in zip(predictions, lower_bounds, upper_bounds) if lower_bound <= prediction <= upper_bound)

# Calculate the percentage within bounds
percentage_within_bounds = (count_within_bounds / len(predictions)) 

# Print the result
print(f"Percentage of predictions within bounds: {percentage_within_bounds}%")


IF att11 = (-inf, 33.50) AND att10 = <49.01, inf) THEN label = {2.9E-83} [-3.9E-67,3.9E-67]
IF att6 = <-0.0083, inf) AND att11 = (-inf, 33.50) AND att10 = <45.50, inf) THEN label = {3.9E-66} [-3.4E-46,3.4E-46]
IF att11 = (-inf, 37.00) AND att10 = <45.50, inf) THEN label = {9.7E-66} [-3.9E-39,3.9E-39]
IF att11 = (-inf, 63.00) AND att10 = <48.01, inf) THEN label = {1.0E-50} [-1.2E-33,1.2E-33]
IF att8 = (-inf, 1.00) AND att11 = (-inf, 63.00) AND att10 = <46.01, inf) AND att3 = <-0.0058, inf) THEN label = {7.0E-47} [-3.0E-30,3.0E-30]
IF att6 = <-0.0013, inf) AND att8 = <6.0E-4, inf) AND att7 = <-0.0034, 1.00) AND att1 = (-inf, 1.00) AND att10 = <47.01, inf) AND att4 = <2.6E-4, inf) THEN label = {1.1E-49} [-1.5E-21,1.5E-21]
IF att11 = (-inf, 64.49) AND att10 = <46.00, inf) THEN label = {2.7E-47} [-5.1E-21,5.1E-21]
IF att11 = (-inf, 90.50) AND att10 = <45.50, inf) THEN label = {7.1E-46} [-2.3E-12,2.3E-12]
IF att10 = <45.50, inf) THEN label = {3.8E-39} [-1.3E-7,1.3E-7]
IF att6 = <9.7E-4, inf)

In [6]:
clf = DecisionTreeRegressor(random_state=0)
clf.fit(samp, yee)
pred = clf.predict(test_set)


# Define your bounds
lower_bounds = [flattened_mean[i] - 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]
upper_bounds = [flattened_mean[i] + 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]

# Initialize a count for predictions within bounds
pr_count_within_bounds = sum(1 for prediction, lower_bound, upper_bound in zip(pred, lower_bounds, upper_bounds) if lower_bound <= prediction <= upper_bound)

# Calculate the percentage within bounds
percentage_within_bounds = (pr_count_within_bounds / len(pred)) 

# Print the result
print(f"Percentage of predictions within bounds: {percentage_within_bounds}%")
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
            p1 += [f"({name} <= {np.round(threshold, 3)})"]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [f"({name} > {np.round(threshold, 3)})"]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = "if "
        
        for p in path[:-1]:
            if rule != "if ":
                rule += " and "
            rule += str(p)
        rule += " then "
        if class_names is None:
            rule += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            rule += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
        rule += f" | based on {path[-1][1]:,} samples"
        rules += [rule]
        
    return rules
rules = get_rules(clf, Xe.columns, None)
for r in rules:
    print(r)
len(rules)

Percentage of predictions within bounds: 0.012%
if (categorical_optimizer_adam > -0.01) and (neurons <= 91.004) and (categorical_batchsize_64 > -0.01) and (categorical_optimizer_rmsprop > -0.009) and (categorical_activation_relu <= 1.01) and (categorical_batchsize_16 <= 0.993) and (categorical_loss_mse <= 1.001) and (neurons <= 87.995) and (categorical_batchsize_32 > -0.01) and (neurons <= 60.501) and (categorical_activation_tanh > -0.007) and (categorical_optimizer_adam <= 1.01) and (categorical_activation_relu > -0.01) and (categorical_batchsize_32 <= 1.006) then response: 0.0 | based on 256 samples
if (categorical_optimizer_adam > -0.01) and (neurons <= 91.004) and (categorical_batchsize_64 > -0.01) and (categorical_optimizer_rmsprop > -0.009) and (categorical_activation_relu <= 1.01) and (categorical_batchsize_16 > 0.993) and (categorical_batchsize_16 > 0.993) and (categorical_batchsize_16 > 0.994) and (epochs > 25.499) and (categorical_activation_tanh > -0.01) and (neurons > 18.0)

136

In [6]:
# from itertools import combinations

# # Given rule list
# rule_list = rules

# # Function to extract features from a rule
# def extract_features(rule):
#     features = set(rule.split("and"))
#     return features

# # Function to calculate Jaccard similarity between two rules
# def jaccard_similarity(rule1, rule2):
#     features1 = extract_features(rule1)
#     features2 = extract_features(rule2)
#     intersection = len(features1.intersection(features2))
#     union = len(features1.union(features2))
    
#     if union == 0:
#         return 0.0  # Handle the case where both sets are empty
#     else:
#         return intersection / union

# # Calculate Jaccard similarity between all pairs of rules and collect them
# similarities = []
# for rule1, rule2 in combinations(rule_list, 2):
#     similarity = jaccard_similarity(rule1, rule2)
#     similarities.append(similarity)

# # Calculate the mean Jaccard similarity for the entire rule set
# mean_similarity = sum(similarities) / len(similarities)

# print(f"Mean Jaccard Similarity for the Rule Set: {mean_similarity:.2f}")

In [17]:
from skrules import SkopeRules
from sklearn.preprocessing import KBinsDiscretizer
yee = yee/np.max(yee)

# transform the dataset with KBinsDiscretizer
enc = KBinsDiscretizer(n_bins=2, encode="ordinal", strategy='kmeans')
Y_binned = enc.fit_transform(yee.reshape(-1, 1))

clf1 = SkopeRules(
    max_depth_duplication=3, max_depth=3, max_features=0.5,
    max_samples_features=0.5, random_state=0, n_estimators=20,
    feature_names=Xe.columns, recall_min=0.04, precision_min=0.6)

clf1.fit(samp, Y_binned)

p = clf.predict(test_set)


# Define your bounds
lower_bounds = [flattened_mean[i] - 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]
upper_bounds = [flattened_mean[i] + 0.5*flattened_mean[i]  for i in range(len(flattened_mean))]

# Initialize a count for predictions within bounds
clf_count_within_bounds = sum(1 for prediction, lower_bound, upper_bound in zip(p, lower_bounds, upper_bounds) if lower_bound <= prediction <= upper_bound)

# Calculate the percentage within bounds
percentage_within_bounds = (clf_count_within_bounds / len(p)) 

# Print the result
print(f"Percentage of predictions within bounds: {percentage_within_bounds}%")

print(len(clf1.rules_))
print(clf1.rules_)

 c:\users\uif75022\phddevstuff\rxboplusdev\rxboplus\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning:A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 c:\users\uif75022\phddevstuff\rxboplusdev\rxboplus\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning:`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
 c:\users\uif75022\phddevstuff\rxboplusdev\rxboplus\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning:`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.


Percentage of predictions within bounds: 0.012%
2
[('patience <= 17.99722194671631 and patience > 17.501938819885254 and categorical_loss_mae > 1.0055921077728271', (1.0, 0.6666666666666666, 2)), ('neurons <= 91.5026969909668 and neurons > 91.00367736816406 and categorical_batchsize_64 > 0.49801159161143005', (0.6666666666666666, 1.0, 2))]
